In [25]:
#find snps
#import data science libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import allel
import numpy as np
import scipy.stats as st
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [36]:
root = "/Users/k23030440/github_projects/S.-Pombe-MLPs/"
snps_file = '/Users/k23030440/Jeffares copy/Jeffares SNPs/ERZ289255/Spombe.2013-01-02.filt3c.nr57-final.snps.anno-snpeff3.cleaned4.vcf'
indels_file = '/Users/k23030440/Jeffares copy/Jeffares SNPs/ERZ289254/Spombe.indels.unique.nr57.final.annotated.2014-01-23.cleaned4.vcf.gz'
final_results_screen = pd.read_csv(root + "/Bence folder/Bioneer screen/final_results_screen.csv")


In [37]:
#genome coordinates for genes
df = pd.read_csv(root + '/external data/Pombase files/Schizosaccharomyces_pombe_all_chromosomes.gff3', sep='\t', header=1)
#break last column into 2 at ;
df[['a','b']] = df["ID=SPAC1002.01;Name=mrx11"].str.split(';',expand=True)
#remove column ID=SPAC1002.01;Name=mrx11
df.drop(columns =["ID=SPAC1002.01;Name=mrx11"], inplace = True)
#remove ID= from column a and rename it to ID
df['ID'] = df['a'].str.replace('ID=', '')
#remove column a
df.drop(columns =['a','PomBase',".","..1"], inplace = True)
#rename b to Parent
df.rename(columns={'b':'Parent','gene':'object','I':'chromosome','1798347':"start",'1798835':"end","+":"strand"}, inplace=True)
#In parent remove anything before =
df['Parent'] = df['Parent'].str.replace('Parent=', '')
#or Name=
df['Parent'] = df['Parent'].str.replace('Name=', '')
#create an aempty column called detail
df['detail'] = ""
#split ID column at :, and if there is something after put it in detail column
df['detail'] = df['ID'].str.split(':', expand=True)[1]
#split ID column at :, and if there is something before put it in ID column
df['ID'] = df['ID'].str.split(':', expand=True)[0]#load in vcf file

In [57]:
df = df.loc[df["object"] == "gene"]
mediator = ["SPAC23H4.17c","SPBC12D12.06","SPAC688.08","SPAC589.02c","SPAC5D6.05","SPCC1450.05c"]

df = df.loc[df["ID"].isin(mediator) | df["ID"].isin(final_results_screen["strain"].values) | df["Parent"].isin(final_results_screen["strain"].values )]
df.reset_index(drop=True, inplace=True)
df

,chromosome,object,start,end,strand,Parent,ID,detail
0,I,gene,2192037,2193615,-,hst4,SPAC1783.04c,None
1,I,gene,648301,649565,-,csk1,SPAC1D4.06c,None
2,I,gene,686535,689179,+,res2,SPAC22F3.09c,None
3,I,gene,1576707,1578028,+,srb10,SPAC23H4.17c,None
4,I,gene,3069322,3071099,-,shm1,SPAC24C9.12c,None
5,I,gene,5131870,5133058,-,rrp17,SPAC29A4.09,None
6,I,gene,2832707,2834985,+,his6,SPAC3F10.09,None
7,I,gene,3095075,3098667,-,med13,SPAC589.02c,None
8,I,gene,1503761,1504867,-,med18,SPAC5D6.05,None
9,I,gene,2039370,2040417,-,None,SPAC607.02c,None


In [39]:
samples

array(['JB1110', 'JB1117', 'JB1154', 'JB1171', 'JB1174', 'JB1180',
       'JB1197', 'JB1205', 'JB1206', 'JB1207', 'JB22', 'JB4', 'JB758',
       'JB760', 'JB762', 'JB837', 'JB838', 'JB840', 'JB841', 'JB842',
       'JB845', 'JB846', 'JB848', 'JB852', 'JB853', 'JB854', 'JB858',
       'JB862', 'JB864', 'JB869', 'JB870', 'JB871', 'JB872', 'JB873',
       'JB874', 'JB875', 'JB878', 'JB879', 'JB884', 'JB899', 'JB900',
       'JB902', 'JB910', 'JB913', 'JB914', 'JB916', 'JB917', 'JB918',
       'JB929', 'JB930', 'JB931', 'JB934', 'JB938', 'JB939', 'JB942',
       'JB943', 'JB953'], dtype=object)

In [58]:
interesting_strains

['JB914', 'JB871', 'JB1117', 'JB872', 'JB934', 'JB848', 'JB953', 'JB869']

In [75]:
snps_data = allel.read_vcf(snps_file,fields=['samples','variants/CHROM','variants/POS','calldata/GT','variants/ALT','variants/REF'])
samples = snps_data['samples']
interesting_strains = samples
interesting_strains = ["JB914","JB871","JB1117", "JB872", "JB934", "JB759", "JB848","JB953","JB869"]

#finding the index of the interesting strains
interesting_strains_index = []
interesting_strains_final = []
for strain in interesting_strains:
    try:
        index = np.where(samples == strain)[0][0]
    except:
        #remove strain from interesting_strains if not found
        
        print(f'Strain {strain} not found in samples, removing from list.')
        continue
    interesting_strains_final.append(strain)
    interesting_strains_index.append(index)

interesting_strains = interesting_strains_final.copy()
calls= snps_data['calldata/GT'][:,:,0]
chromosomes = snps_data['variants/CHROM']
positions = snps_data['variants/POS']
alt = snps_data['variants/ALT']
ref= snps_data['variants/REF']


Strain JB759 not found in samples, removing from list.


In [77]:
#iterate through positions
#results is a df with columns: alleles	n	sum_x	y_transpose_x	beta	standard_error	t_stat	p_value	type	allele1	allele2	chromosome	position
results = pd.DataFrame(columns=["strain","ID","chromosome","position","gene","type","allele1","allele2","p_value"])
#rough gene coordinates
#take mean of start and end
rough_gene_coordinates = (df["start"]+df["end"])/2


In [78]:

for i in range(len(positions)):
      #if positions[i] is not near to any of the rough_gene_coordinates  +-10000
      
      if np.isclose(positions[i],rough_gene_coordinates,atol=20000).any() == False:
            
            continue
      for j in range(len(df)):
            #for + strand
            if positions[i]<(df["end"][j]+200) and positions[i]>(df["start"][j]-200) and chromosomes[i]==df["chromosome"][j]:
                  #iterate through interesting strains
                  for k in range(len(interesting_strains_index)):
                        #take the value of the genotype of the interesting strain at the position
                        call_value = calls[i][interesting_strains_index[k]]
                        #if this value is 0 or 1, and there is only max 2 strains with this value, then add the strain name to the detail column
                        if call_value == 0 or call_value == 1 or call_value == 2:
                              if sum(calls[i]==call_value) <= 5:
                                    
                                    #results = results.append({"strain":interesting_strains[k],"ID":df["ID"][j],"chromosome":chromosomes[i],"position":positions[i], "gene":df["Parent"][j],"type":"snp","allele1":ref[i],"allele2":alt[i][call_value-1],"p_value":0.05},ignore_index=True)
                                    results = pd.concat([results,pd.DataFrame({"strain":interesting_strains[k],"ID":df["ID"][j],"chromosome":chromosomes[i],"position":positions[i], "gene":df["Parent"][j],"type":"snp","allele1":ref[i],"allele2":alt[i][call_value-1],"p_value":0.05},index=[0])],ignore_index=True)


In [79]:
indels_data = allel.read_vcf(indels_file,fields=['samples','variants/CHROM','variants/POS','calldata/GT','variants/ALT','variants/REF'])

calls= indels_data['calldata/GT'][:,:,0]
chromosomes = indels_data['variants/CHROM']
positions = indels_data['variants/POS']
alt = indels_data['variants/ALT']
ref= indels_data['variants/REF']


In [80]:

for i in range(len(positions)):
   
      if np.isclose(positions[i],rough_gene_coordinates,atol=20000).any() == False:
            continue
      for j in range(len(df)):
            #for + strand
            if positions[i]<(df["end"][j]+200) and positions[i]>(df["start"][j]-200) and chromosomes[i]==df["chromosome"][j]:
                  #iterate through interesting strains
                  for k in range(len(interesting_strains_index)):
                        #take the value of the genotype of the interesting strain at the position
                        call_value = calls[i][interesting_strains_index[k]]
                        #if this value is 0 or 1, and there is only max 3 strains with this value, then add the strain name to the detail column
                        if call_value == 0 or call_value == 1 or call_value == 2 or call_value == 3:
                              if sum(calls[i]==call_value) <= 3:
                                    
                                    results = pd.concat([results,pd.DataFrame({"strain":interesting_strains[k],"ID":df["ID"][j],"chromosome":chromosomes[i],"position":positions[i], "gene":df["Parent"][j],"type":"indel","allele1":ref[i],"allele2":alt[i][call_value-1],"p_value":0.05},index=[0])],ignore_index=True)


In [83]:
results.tail(50)

,strain,ID,chromosome,position,gene,type,allele1,allele2,p_value
208,JB953,SPCC895.05,III,1004996,for3,snp,T,C,0.05
209,JB953,SPCC895.05,III,1005353,for3,snp,C,T,0.05
210,JB953,SPCC895.05,III,1005518,for3,snp,C,T,0.05
211,JB914,SPCC895.05,III,1006171,for3,snp,C,T,0.05
212,JB953,SPCC895.05,III,1006799,for3,snp,C,T,0.05
213,JB871,SPCC1259.03,III,1040021,rpa12,snp,C,T,0.05
214,JB953,SPCC1259.03,III,1040021,rpa12,snp,C,T,0.05
215,JB953,SPCC1259.03,III,1040280,rpa12,snp,G,T,0.05
216,JB953,SPCC1259.03,III,1040452,rpa12,snp,C,A,0.05
217,JB953,SPCC1259.03,III,1040467,rpa12,snp,G,A,0.05


In [84]:
#make csv file
results.to_csv("/Users/k23030440/github_projects/S.-Pombe-MLPs/Bence folder/Looking for variants in wild isolates/snp_hunting.csv", index=False)

In [85]:
results.tail(5)


,strain,ID,chromosome,position,gene,type,allele1,allele2,p_value
253,JB848,SPBC23G7.12c,II,2122791,rpt6,indel,T,TA,0.05
254,JB914,SPBC12D12.06,II,2316851,srb11,indel,TA,T,0.05
255,JB869,SPBC12D12.06,II,2316851,srb11,indel,TA,T,0.05
256,JB871,SPBC405.07,II,3153756,rpl3602,indel,T,TTA,0.05
257,JB953,SPBC16G5.15c,II,4239834,fkh2,indel,T,TA,0.05


In [ ]:
%load_ext watermark
%watermark


The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2023-11-03T18:58:41.209602+00:00

Python implementation: CPython
Python version       : 3.9.0
IPython version      : 7.31.1

Compiler    : Clang 11.0.0 
OS          : Darwin
Release     : 21.5.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit



In [ ]:
%watermark --iversions


numpy     : 1.23.4
matplotlib: 3.8.0
allel     : 1.3.5
pandas    : 1.4.4
bokeh     : 3.2.2
scipy     : 1.11.3
iqplot    : 0.3.3

